In [1]:
import cv2
import imageio.v3 as iio
import numpy as np
import read_roi 
from read_roi import read_roi_file
import os
import matplotlib.pyplot as plt
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from shapely.geometry import *
import concurrent.futures

In [2]:
data_dir = "/media/molloi-lab/2TB/BAC/Clean_Dataset_full"

In [119]:
# data_list = ["103669", "103059", "103959", "106466", "109462", "112826", "120691", "134480"]
data_list = ["103959"]
for i in range(len(data_list)):
    data_list[i] = data_dir + "/SID-" + data_list[i]
data_list

['/media/molloi-lab/2TB/BAC/Clean_Dataset_full/SID-103959']

<!-- Files that need to be double-check:
    C:\BAC_dataset\collected_dataset_for_ML\train\label\SID-101237\1.2.840.113681.2216859912.974.3569244225.82.roi  
    C:\BAC_dataset\collected_dataset_for_ML\train\label\SID-102318\1.2.840.113681.2230563618.924.3633869514.120.roi  
    C:\BAC_dataset\collected_dataset_for_ML\train\label\SID-102318\1.2.840.113681.2230563618.924.3633869514.124.roi  
    C:\BAC_dataset\collected_dataset_for_ML\train\label\SID-102442\1.2.840.113681.2230563618.1412.3560865020.102.roi  
    C:\BAC_dataset\collected_dataset_for_ML\train\label\SID-104835\1.2.840.113681.2229463144.937.3560426696.121.roi  
    C:\BAC_dataset\collected_dataset_for_ML\train\label\SID-104902\1.2.840.113619.2.255.273315310659.2732131022111926.149.roi  
    C:\BAC_dataset\collected_dataset_for_ML\train\label\SID-105834\1.2.840.113681.2230565808.1039.3564316702.109.roi  
    C:\BAC_dataset\collected_dataset_for_ML\train\label\SID-105834\1.2.840.113681.2230565808.1039.3564316702.113.roi  
    C:\BAC_dataset\collected_dataset_for_ML\train\label\SID-106494\1.2.840.113681.2229444882.954.3632752492.113.roi  
    C:\BAC_dataset\collected_dataset_for_ML\train\label\SID-106494\1.2.840.113681.2229444882.954.3632752492.117.roi  
    C:\BAC_dataset\collected_dataset_for_ML\train\label\SID-107720\1.2.840.113681.2230563618.920.3573734591.199.roi  
    C:\BAC_dataset\collected_dataset_for_ML\train\label\SID-107720\1.2.840.113681.2230563618.920.3573734591.201.roi  
    C:\BAC_dataset\collected_dataset_for_ML\train\label\SID-108396\1.2.840.113681.2230565084.928.3546255633.58.roi  
    C:\BAC_dataset\collected_dataset_for_ML\train\label\SID-108747\1.2.840.113619.2.66.2211293102.1691130530110201.149.roi  
    C:\BAC_dataset\collected_dataset_for_ML\train\label\SID-109826\1.2.840.113619.2.255.207924715888.2421130717172050.505.roi  
    C:\BAC_dataset\collected_dataset_for_ML\train\label\SID-110502\1.2.840.113619.2.255.207924715888.2163130701110032.249.roi  
    C:\BAC_dataset\collected_dataset_for_ML\train\label\SID-110751\1.2.840.113619.2.255.207924715888.2700130722105839.205.roi  
    C:\BAC_dataset\collected_dataset_for_ML\train\label\SID-110852\1.2.840.113619.2.255.207924750210.2991130722104520.125.roi  
    C:\BAC_dataset\collected_dataset_for_ML\train\label\SID-112361\1.2.840.113681.2230565808.948.3553858125.196.roi  
    C:\BAC_dataset\collected_dataset_for_ML\train\label\SID-112361\1.2.840.113681.2230565808.948.3553858125.198.roi    
    C:\BAC_dataset\collected_dataset_for_ML\train\label\SID-112361\1.2.840.113681.2230565808.948.3553858125.200.roi  
    C:\BAC_dataset\collected_dataset_for_ML\train\label\SID-112361\1.2.840.113681.2230565808.948.3553858125.202.roi    
    C:\BAC_dataset\collected_dataset_for_ML\train\label\SID-112723\1.2.840.113619.2.255.273316596199.3153130827141454.277.roi  
    C:\BAC_dataset\collected_dataset_for_ML\train\label\SID-112732\1.2.840.113681.2230565084.940.3554981006.445.roi  
    C:\BAC_dataset\collected_dataset_for_ML\train\label\SID-112732\1.2.840.113681.2230565084.940.3554981006.447.roi   -->

In [141]:
def sort_list(list1, list2):
    zipped_pairs = zip(list2, list1)
    z = [x for _, x in sorted(zipped_pairs)]
    return z

def main(curr_dir):
    for foldername, subfolders, filenames in os.walk(curr_dir):
        for filename in filenames:
            if filename.endswith("roi"):
                UID = os.path.splitext(filename)[0]
                png_file_name = UID + ".png"
                # print(png_file_name)
                png_path = os.path.join(foldername, png_file_name)
                roi_path = os.path.join(foldername, filename)
                if not os.path.isfile(png_path):
                    print("Not found: "+png_path)
                else:
                    # read png
                    lbl = iio.imread(png_path)
                    lbl = np.zeros(lbl.shape)
                    # read roi
                    roi = read_roi_file(roi_path)[UID]
                    if "paths" in roi.keys():
                        # read "paths"
                        paths = roi["paths"]

                        prev_polys = []
                        for ps in paths:
                            pts = np.array(ps, np.int32)


                            # prev_pt = pts[len(pts)-1]
                            # for (i,pt) in enumerate(pts):
                            #     cv2.line(lbl, prev_pt, pt, 50+205*i/len(pts), 2) 
                            #     prev_pt = pt

                            cv2.fillPoly(lbl, [pts], 255)
                            curr_poly = Polygon(ps)
                            curr_area = curr_poly.area
                            for (i, prev_poly) in enumerate(prev_polys):
                                if prev_poly.intersects(curr_poly):
                                    prev_area = prev_poly.area
                                    common_area = prev_poly.intersection(curr_poly).area
                                    if common_area == prev_area:
                                        prev_pts = np.array(prev_poly.exterior.coords, np.int32)
                                        cv2.fillPoly(lbl, [prev_pts], 0)
                                    if common_area == curr_area:
                                        cv2.fillPoly(lbl, [pts], 0)
                                        break
                            prev_polys.append(curr_poly)
                        cv2.imwrite(png_path, lbl)
                    else:
                        # read "x" and "y"
                        x = roi["x"]
                        y = roi["y"]
                        pts = np.array(list(zip(x, y)), np.int32)
                        cv2.fillPoly(lbl, [pts], 255) # 255 stands for white color
                        cv2.imwrite(png_path, lbl)

def process_chunk(chunk):
    for directory in chunk:
        main(directory)

def split_list_into_chunks(lst, chunk_size):
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]


In [142]:
SIDs = os.listdir(data_dir)
directories = []
for sid in SIDs:
    directories.append(data_dir + "/" + sid)

num_threads = 64

chunk_size = len(directories) // num_threads
directory_chunks = list(split_list_into_chunks(directories, chunk_size))

with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Process each chunk concurrently
    executor.map(process_chunk, directory_chunks)